In [14]:
import pandas as pd
import numpy as np

# define the name of the csv file you want to read in
accounts = './data/accounts.csv'
playbacks = './data/playbacks.csv'
subscriptions = './data/subscriptions.csv'

# PLAYBACKS

In [15]:
# Read playbacks
df_playbacks = pd.read_csv(playbacks)
df_playbacks.columns = df_playbacks.columns.str.lower()
display(df_playbacks.shape)
display(df_playbacks.date_start.tail())
# one account can have multiple subscriptions
# what about pausing the movie
# IP + User_Agent-> check for multiple persons per subscriptions
    # IP/Account
# average playback duration
# cancelled session?

(139569, 7)

139564    2020-10-21 13:21:30
139565    2020-10-21 12:35:04
139566    2020-10-21 12:05:14
139567    2020-10-21 11:46:11
139568    2020-10-21 11:18:58
Name: date_start, dtype: object

In [16]:
df_playbacks.isna().sum()

subscription_key    0
account_key         0
movie_id            0
date_start          0
playback_time       0
user_agent          0
ip_hash             0
dtype: int64

In [17]:
df_playbacks.describe()
# have a closer look at max playback times

,movie_id,playback_time
count,139569.000000,139569.00000
mean,712.801589,6442.57679
std,346.698325,8191.70516
min,8.000000,960.00000
25%,453.000000,4680.00000
50%,745.000000,5760.00000
75%,977.000000,6660.00000
max,1344.000000,472920.00000


In [19]:
#DIFFERENTIATION BETWEEN DESKTOP/MOBILE
#TODO Create 'DESKTOP vs. MOBILE' column
#TODO filmingo app users dedicated column?!

#df_playbacks.user_agent.count()
#5387 unique values
#139.569 count

#SEARCH FOR 'Macintosh'
#df_playbacks[df_playbacks.user_agent.str.contains('Macintosh', na=False)].user_agent.count()
#64.401 count ~50%

#SEARCH FOR 'Windows'
#df_playbacks[df_playbacks.user_agent.str.contains('Windows', na=False)].user_agent.count()
#41.475

#DESKTOP Views
df_playbacks[df_playbacks.user_agent.str.contains(('Windows|Macintosh|TV|Linux'), na=False)].user_agent.count()
#117.081 Desktop playbacks

# Mobile Views
#df_playbacks[df_playbacks.user_agent.str.contains(('Android|iOS|iPhone|iPad'), na=False)].user_agent.count()
# 22.800 Mobile views

df_playbacks[df_playbacks.user_agent.str.contains(('Linux'), na=False)].user_agent.unique()
df_playbacks[df_playbacks.user_agent.str.contains(('Darwin'), na=False)].user_agent.count()

#HOW MANY PEOPLE ARE USING THE APP
df_playbacks[~df_playbacks.user_agent.str.contains('filmingo', na=False)].user_agent.count()
#13.173 playbacks via the app



126396

In [27]:
#Let's create the new columns
df_playbacks.loc[df_playbacks['user_agent'].str.contains('Windows|Macintosh|TV|Linux|Darwin|CrOS'), 'device'] = 'desktop'
df_playbacks.loc[df_playbacks['user_agent'].str.contains('Android|iOS|iPhone|iPad'), 'device'] = 'mobile'
df_playbacks.loc[df_playbacks['user_agent'].str.contains('filmingo'), 'app user'] = 'yes'
df_playbacks.loc[~df_playbacks['user_agent'].str.contains('filmingo'), 'app user'] = 'no'
df_playbacks.groupby('device').count()
#df_playbacks[~df_playbacks.device.str.contains(('Android|iOS|iPhone|iPad|Windows|Macintosh|TV|Linux'), na=False)].user_agent.count()



,subscription_key,account_key,movie_id,date_start,playback_time,user_agent,ip_hash,app user
device,,,,,,,,
desktop,116734,116734,116734,116734,116734,116734,116734,116734
mobile,22800,22800,22800,22800,22800,22800,22800,22800


In [28]:
df_playbacks.head()

,subscription_key,account_key,movie_id,date_start,playback_time,user_agent,ip_hash,device,app user
0,9679626b6cc1765f8bc8da98fda2d8e8f28f40736fd777...,024ebde74e23ca924cff0a3989e7f87efe2b2cbcd7ad2c...,1189,2022-09-30 23:55:31,12420,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,0b14aa31d5635ee899be928089fd810c655a7ec7785911...,desktop,no
1,513d006472c99ef047c64bafad57a39906ad5c618505b1...,8968e3851322a89c265a0ec4c81341361e3b0953502d65...,1327,2022-09-30 23:48:36,1440,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4...,c65100e7d48b9e06ca6d89ffe594066553ea63ddc70ee1...,desktop,no
2,cac0c77d3aedc55303b8dfea314052eba5dcf4817e4962...,8a2d7ac64595778ed5fe7edb2dd43f843f9879d773df1a...,212,2022-09-30 23:47:33,1080,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,db17d38bf724c377dcd463f7ad8df8279b6122c8890773...,desktop,no
3,a35503448084642ac8651a0f482db410ccdd7c7716df1d...,7f654ed491a8d1c5b6eb54ea98f6b30b0f9d9515aaad22...,108,2022-09-30 23:46:52,5400,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,69839531a94bf2468744c083a7bf4b78f39c8b1b299c34...,desktop,no
4,ff97a5e4ea0bb411f72e2e1750fb94b248ec2026406a3f...,6efdbec56858f25686797c974bfa67809a718408bd3631...,1314,2022-09-30 23:16:49,10380,"filmingo v2.1 (83); iOS 15.6.1; iPad8,11",652d233c8526ebefb7a8e939ca546fe1a76498775b682a...,mobile,yes


# EDA part

In [38]:
import sql_functions as sf
schema = 'capstone_filmingo'

# One Time Rentals per account

In [46]:
sf.get_dataframe(f' SELECT onetime_rental_count FROM {schema}.accounts').describe()

,onetime_rental_count
count,22154.000000
mean,1.300442
std,3.232544
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,106.000000


# Subscriptions per account

In [47]:
sf.get_dataframe(f' SELECT subscription_count FROM {schema}.accounts').describe()

,subscription_count
count,22154.000000
mean,0.550465
std,0.830078
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,13.000000


# Total playbacks per year

In [78]:
sf.get_dataframe(f'SELECT EXTRACT(year FROM date_start) as year, account_key as account, count(*) as playbacks FROM {schema}.playbacks GROUP BY account, EXTRACT(year FROM date_start)').head(15)

,year,account,playbacks
0,2020.0,0017950cb0d712c53942f0d58192b369aa815ddc128f90...,2
1,2021.0,0017950cb0d712c53942f0d58192b369aa815ddc128f90...,13
2,2022.0,0017950cb0d712c53942f0d58192b369aa815ddc128f90...,7
3,2021.0,001cb20257aaf26f10ad88aaa5f8ef6ab84b84431cab4d...,8
4,2020.0,0023272b55a079432360e20d920e11039cfdb0e831938e...,1
5,2021.0,0028522c0f5134803687e5015b0621a9c81855174a4a7f...,1
6,2021.0,0036ef0e8bfe9d8b7feef88b872b1a61ef1aa3afed2056...,2
7,2022.0,0036ef0e8bfe9d8b7feef88b872b1a61ef1aa3afed2056...,10
8,2021.0,0040bc486ad497cb1daecd82d18979fff24e87edbbce28...,3
9,2022.0,0040bc486ad497cb1daecd82d18979fff24e87edbbce28...,5


# Playbacks per device

In [107]:
sf.get_dataframe(f' SELECT device, COUNT(*) as playbacks FROM {schema}.playbacks GROUP BY device').head()

,device,playbacks
0,desktop,116769
1,mobile,22800


In [93]:
sf.get_dataframe(f' SELECT app_user, COUNT(*) FROM {schema}.playbacks GROUP BY app_user').head()

,app_user,count
0,no,126396
1,yes,13173


In [94]:
sf.get_dataframe(f' SELECT device, app_user, COUNT(*) FROM {schema}.playbacks GROUP BY device, app_user').head()

#TODO rethink the categorization

,device,app_user,count
0,desktop,no,116769
1,mobile,no,9627
2,mobile,yes,13173


In [ ]:
#TODO Ugh not working rn
sf.get_dataframe(f" SELECT user_agent, COUNT(*) FROM {schema}.playbacks WHERE user_agent LIKE '%%iPhone%%'").head()


# Playbacks per subscription type (per month)

In [125]:
#sf.get_dataframe(f' SELECT device, COUNT(*) as playbacks FROM {schema}.playbacks GROUP BY device').head()
sf.get_dataframe(f' SELECT subscription_type, COUNT(*) as playbacks FROM {schema}.playbacks INNER JOIN {schema}.subscriptions on {schema}.playbacks.subscription_key = {schema}.subscriptions.subscription_key GROUP BY subscription_type').head()

,subscription_type,playbacks
0,BASIC,48604
1,PATRON,41737
2,STANDARD,49228


# Playbacks per month/weekday 

In [134]:
sf.get_dataframe(f' SELECT EXTRACT(year FROM date_start) as year, subscription_type, COUNT(*) as playbacks FROM {schema}.playbacks INNER JOIN {schema}.subscriptions on {schema}.playbacks.subscription_key = {schema}.subscriptions.subscription_key GROUP BY subscription_type, year ORDER BY subscription_type').head(30)

,year,subscription_type,playbacks
0,2020.0,BASIC,5565
1,2021.0,BASIC,27888
2,2022.0,BASIC,15151
3,2020.0,PATRON,5009
4,2021.0,PATRON,24168
5,2022.0,PATRON,12560
6,2020.0,STANDARD,8354
7,2021.0,STANDARD,28671
8,2022.0,STANDARD,12203


In [137]:
sf.get_dataframe(f' SELECT EXTRACT(year FROM date_start) as year, EXTRACT(month FROM date_start) as month, subscription_type, COUNT(*) as playbacks FROM {schema}.playbacks INNER JOIN {schema}.subscriptions on {schema}.playbacks.subscription_key = {schema}.subscriptions.subscription_key GROUP BY subscription_type, year, month ORDER BY year, month').head(30)

,year,month,subscription_type,playbacks
0,2020.0,10.0,BASIC,884
1,2020.0,10.0,PATRON,627
2,2020.0,10.0,STANDARD,909
3,2020.0,11.0,BASIC,2232
4,2020.0,11.0,PATRON,2032
5,2020.0,11.0,STANDARD,4108
6,2020.0,12.0,BASIC,2449
7,2020.0,12.0,PATRON,2350
8,2020.0,12.0,STANDARD,3337
9,2021.0,1.0,BASIC,3234


# Playbacks per account (per month)

In [144]:
sf.get_dataframe(f' SELECT EXTRACT(year FROM date_start) as year, EXTRACT(month FROM date_start) as month, playbacks.account_key as account, COUNT(*) as playbacks FROM {schema}.playbacks INNER JOIN {schema}.accounts on {schema}.playbacks.account_key = {schema}.accounts.account_key GROUP BY playbacks.account_key, year, month ORDER BY year, month').head(30)

,year,month,account,playbacks
0,2020.0,10.0,00b3ca364f49f801f257e1e6dff4810cbefb34b3ff4780...,2
1,2020.0,10.0,00c7287a5449e9b4071da68e49053a507676e09b8a15ee...,1
2,2020.0,10.0,00ed08533436943cbe509eb354af609105ab88a65aec0c...,3
3,2020.0,10.0,0113a563f7147636f69eb112b0a8b04dac44259e78818f...,2
4,2020.0,10.0,012d4f225b098eca174635e86f9070673cf324b8eb5bd8...,3
5,2020.0,10.0,013918430a268342345f74d53fde521bfc41ceca6ab279...,2
6,2020.0,10.0,0140f3db0368d382ab9891407ebffb467ad37d11e2abc6...,2
7,2020.0,10.0,01932e526d5ee6584b257c17f558375d6607f51cb21d54...,1
8,2020.0,10.0,01c7fff04676899cf2feaea3e31becb496732b09819dbf...,2
9,2020.0,10.0,01da5c23a89401494df2cc819721f040713f0cbd581ef6...,1


# Hours watched per account (per month)

In [152]:
sf.get_dataframe(f' SELECT EXTRACT(year FROM date_start) as year, EXTRACT(month FROM date_start) as month, playbacks.account_key as account, (SUM(playback_time)/60/60) as playback_hours FROM {schema}.playbacks INNER JOIN {schema}.accounts on {schema}.playbacks.account_key = {schema}.accounts.account_key GROUP BY playbacks.account_key, year, month ORDER BY year, month').head(30)

,year,month,account,playback_hours
0,2020.0,10.0,00b3ca364f49f801f257e1e6dff4810cbefb34b3ff4780...,3.750000
1,2020.0,10.0,00c7287a5449e9b4071da68e49053a507676e09b8a15ee...,1.350000
2,2020.0,10.0,00ed08533436943cbe509eb354af609105ab88a65aec0c...,4.050000
3,2020.0,10.0,0113a563f7147636f69eb112b0a8b04dac44259e78818f...,5.250000
4,2020.0,10.0,012d4f225b098eca174635e86f9070673cf324b8eb5bd8...,5.000000
5,2020.0,10.0,013918430a268342345f74d53fde521bfc41ceca6ab279...,2.250000
6,2020.0,10.0,0140f3db0368d382ab9891407ebffb467ad37d11e2abc6...,3.350000
7,2020.0,10.0,01932e526d5ee6584b257c17f558375d6607f51cb21d54...,1.450000
8,2020.0,10.0,01c7fff04676899cf2feaea3e31becb496732b09819dbf...,3.650000
9,2020.0,10.0,01da5c23a89401494df2cc819721f040713f0cbd581ef6...,1.700000


# Sent movie vouchers
How many movie vouchers are being gifted per month / per subscription type?

In [40]:
sf.get_dataframe(f' SELECT DISTINCT * FROM {schema}.playbacks WHERE device IS NULL')

,playback_ID,subscription_key,account_key,movie_id,date_start,playback_time,user_agent,ip_hash,device,app user
